# Data Exploration for Weekly Cash Flow Forecasting

This notebook explores the clean transaction data and prepares it for weekly net cash flow forecasting.

In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

## 1. Load Clean Data

In [2]:
df = pd.read_csv('../processed_data/clean_transactions.csv')
print(f"Data shape: {df.shape}")
df.head()

Data shape: (84528, 11)


C:\Users\asus\AppData\Local\Temp\ipykernel_26112\3876330744.py:1: DtypeWarning: Columns (4) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../processed_data/clean_transactions.csv')


,Name,Period,Account,PK,Offst.acct,Name of offsetting account,Pstng Date,Doc..Date,Amount in USD,LCurr,Category
0,TW10,8,19500100,50,19500102,House Bank 1 - Clearing - EFT Payments,8/1/2025,8/1/25 0:00,-13.85330,TWD,AP
1,TW10,8,19500100,50,19500102,House Bank 1 - Clearing - EFT Payments,8/1/2025,8/1/25 0:00,-15.80906,TWD,AP
2,TW10,8,19500100,50,19500102,House Bank 1 - Clearing - EFT Payments,8/1/2025,8/1/25 0:00,-14.99416,TWD,AP
3,TW10,8,19500100,50,19500102,House Bank 1 - Clearing - EFT Payments,8/1/2025,8/1/25 0:00,-11.89754,TWD,AP
4,TW10,8,19500100,50,19500102,House Bank 1 - Clearing - EFT Payments,8/1/2025,8/1/25 0:00,-15.15714,TWD,AP


## 2. Parse Dates and Explore Date Ranges by Period

In [3]:
# Convert Pstng Date to datetime
df['Pstng Date'] = pd.to_datetime(df['Pstng Date'], format='mixed')
print(f"Date range: {df['Pstng Date'].min()} to {df['Pstng Date'].max()}")

Date range: 2025-01-02 00:00:00 to 2025-10-31 00:00:00


In [4]:
# Get date range for each Period
period_date_ranges = df.groupby('Period').agg(
    Min_Date=('Pstng Date', 'min'),
    Max_Date=('Pstng Date', 'max'),
    Count=('Pstng Date', 'count'),
    Total_Amount=('Amount in USD', 'sum')
).reset_index()

period_date_ranges['Days_Span'] = (period_date_ranges['Max_Date'] - period_date_ranges['Min_Date']).dt.days + 1
print("Date Range for Each Period:")
display(period_date_ranges)

Date Range for Each Period:


,Period,Min_Date,Max_Date,Count,Total_Amount,Days_Span
0,1,2025-01-02,2025-01-31,7270,-7.520905e+05,30
1,2,2025-02-01,2025-02-28,7386,-2.943717e+06,28
2,3,2025-03-01,2025-03-31,8453,-4.931574e+05,31
3,4,2025-04-03,2025-04-30,7141,-2.047720e+06,28
4,5,2025-05-02,2025-05-31,8705,2.945638e+06,30
5,6,2025-06-01,2025-06-30,8410,1.436173e+06,30
6,7,2025-07-03,2025-07-31,8748,-2.126568e+06,29
7,8,2025-08-01,2025-08-31,9703,1.264512e+06,31
8,9,2025-09-01,2025-09-29,9047,1.628147e+05,29
9,10,2025-10-02,2025-10-31,9665,-1.363325e+06,30


## 3. Create Week Number Column for Weekly Aggregation

In [5]:
# Extract week-related columns
df['Year'] = df['Pstng Date'].dt.year
df['Month'] = df['Pstng Date'].dt.month
df['Week'] = df['Pstng Date'].dt.isocalendar().week
df['Year_Week'] = df['Pstng Date'].dt.strftime('%Y-W%V')  # ISO week format
df['Week_Start'] = df['Pstng Date'] - pd.to_timedelta(df['Pstng Date'].dt.dayofweek, unit='D')

print("Columns added: Year, Month, Week, Year_Week, Week_Start")
df[['Pstng Date', 'Year', 'Month', 'Week', 'Year_Week', 'Week_Start']].head(10)

Columns added: Year, Month, Week, Year_Week, Week_Start


,Pstng Date,Year,Month,Week,Year_Week,Week_Start
0,2025-08-01,2025,8,31,2025-W31,2025-07-28
1,2025-08-01,2025,8,31,2025-W31,2025-07-28
2,2025-08-01,2025,8,31,2025-W31,2025-07-28
3,2025-08-01,2025,8,31,2025-W31,2025-07-28
4,2025-08-01,2025,8,31,2025-W31,2025-07-28
5,2025-08-01,2025,8,31,2025-W31,2025-07-28
6,2025-08-01,2025,8,31,2025-W31,2025-07-28
7,2025-08-01,2025,8,31,2025-W31,2025-07-28
8,2025-08-02,2025,8,31,2025-W31,2025-07-28
9,2025-08-02,2025,8,31,2025-W31,2025-07-28


In [6]:
# Number of unique weeks
print(f"Total unique weeks: {df['Year_Week'].nunique()}")
print(f"\nWeeks in data:")
print(df['Year_Week'].value_counts().sort_index())

Total unique weeks: 44

Weeks in data:
Year_Week
2025-W01     962
2025-W02    1695
2025-W03    2623
2025-W04     960
2025-W05    1600
2025-W06    1753
2025-W07    1971
2025-W08    1981
2025-W09    1770
2025-W10    1765
2025-W11    1621
2025-W12    1884
2025-W13    1896
2025-W14    1650
2025-W15    1616
2025-W16    2294
2025-W17    1484
2025-W18    1573
2025-W19    2223
2025-W20    1873
2025-W21    2120
2025-W22    1798
2025-W23    1718
2025-W24    2116
2025-W25    2250
2025-W26    1799
2025-W27    2005
2025-W28    1935
2025-W29    1852
2025-W30    2533
2025-W31    1695
2025-W32    2277
2025-W33    2163
2025-W34    1937
2025-W35    2424
2025-W36    2004
2025-W37    2176
2025-W38    2282
2025-W39    2213
2025-W40    1811
2025-W41    2112
2025-W42    2143
2025-W43    2684
2025-W44    1287
Name: count, dtype: int64


## 4. Classify Inflows and Outflows

In [7]:
# PK 40 = Debit (Inflows), PK 50 = Credit (Outflows)
# Also, positive amounts = inflow, negative = outflow
df['Flow_Type'] = np.where(df['Amount in USD'] > 0, 'Inflow', 'Outflow')

print("Flow Type Distribution:")
print(df['Flow_Type'].value_counts())

Flow Type Distribution:
Flow_Type
Outflow    80662
Inflow      3866
Name: count, dtype: int64


In [8]:
# Create absolute amount for easier aggregation
df['Amount_Abs'] = df['Amount in USD'].abs()

# Summary by Flow Type
flow_summary = df.groupby('Flow_Type').agg(
    Count=('Amount in USD', 'count'),
    Total=('Amount in USD', 'sum'),
    Avg=('Amount_Abs', 'mean')
)
display(flow_summary)

,Count,Total,Avg
Flow_Type,,,
Inflow,3866,1.272657e+08,32919.211620
Outflow,80662,-1.311831e+08,1626.331001


## 5. Weekly Net Cash Flow Aggregation

In [9]:
# Aggregate by week
weekly_cashflow = df.groupby('Week_Start').agg(
    Inflow=('Amount in USD', lambda x: x[x > 0].sum()),
    Outflow=('Amount in USD', lambda x: x[x < 0].sum()),
    Net_Cashflow=('Amount in USD', 'sum'),
    Transaction_Count=('Amount in USD', 'count')
).reset_index()

weekly_cashflow['Outflow_Abs'] = weekly_cashflow['Outflow'].abs()

print(f"Weekly cashflow data: {len(weekly_cashflow)} weeks")
display(weekly_cashflow)

Weekly cashflow data: 44 weeks


,Week_Start,Inflow,Outflow,Net_Cashflow,Transaction_Count,Outflow_Abs
0,2024-12-30,2.346929e+05,-2.031987e+05,3.149420e+04,962,2.031987e+05
1,2025-01-06,5.955187e+05,-7.764411e+05,-1.809224e+05,1695,7.764411e+05
2,2025-01-13,7.727681e+05,-1.237385e+06,-4.646169e+05,2623,1.237385e+06
3,2025-01-20,7.906185e+05,-2.974544e+06,-2.183926e+06,960,2.974544e+06
4,2025-01-27,7.977065e+06,-5.742131e+06,2.234934e+06,1600,5.742131e+06
5,2025-02-03,4.393217e+05,-5.173659e+05,-7.804418e+04,1753,5.173659e+05
6,2025-02-10,5.686534e+05,-8.796654e+05,-3.110121e+05,1971,8.796654e+05
7,2025-02-17,7.293001e+05,-1.990836e+06,-1.261536e+06,1981,1.990836e+06
8,2025-02-24,1.028796e+07,-9.505264e+06,7.826948e+05,1770,9.505264e+06
9,2025-03-03,7.911086e+04,-4.739756e+05,-3.948648e+05,1765,4.739756e+05


In [10]:
# Summary statistics for weekly net cash flow
print("Weekly Net Cash Flow Statistics:")
print(weekly_cashflow['Net_Cashflow'].describe())

Weekly Net Cash Flow Statistics:
count    4.400000e+01
mean    -8.903271e+04
std      3.579555e+06
min     -9.510155e+06
25%     -5.781421e+05
50%     -2.709035e+05
75%      2.672882e+05
max      7.805282e+06
Name: Net_Cashflow, dtype: float64


## 6. Weekly Cash Flow by Category

In [11]:
# Category breakdown
print("Categories in data:")
print(df['Category'].value_counts())

Categories in data:
Category
AP                                   70456
Bank charges                          8395
AR                                    2280
Tax payable                           1168
Other receipt                          968
Custom and Duty                        491
Payroll                                280
Netting AP                             113
Statutory contribution                 108
Interest charges                        56
Netting AR                              51
Loan receipt                            51
Non Netting AR                          38
Loan payment                            19
Non Netting AP                          15
Interest income                         15
Other                                   12
Loan payment and interest charges       10
Dividend payout                          2
Name: count, dtype: int64


In [12]:
# Weekly cashflow by category
weekly_by_category = df.groupby(['Week_Start', 'Category']).agg(
    Amount=('Amount in USD', 'sum'),
    Count=('Amount in USD', 'count')
).reset_index()

# Pivot to get categories as columns
weekly_pivot = weekly_by_category.pivot_table(
    index='Week_Start', 
    columns='Category', 
    values='Amount', 
    aggfunc='sum',
    fill_value=0
).reset_index()

print(f"Weekly cashflow by category: {len(weekly_pivot)} weeks")
display(weekly_pivot.head(10))

Weekly cashflow by category: 44 weeks


Category,Week_Start,AP,AR,Bank charges,Custom and Duty,Dividend payout,Interest charges,Interest income,Loan payment,Loan payment and interest charges,Loan receipt,Netting AP,Netting AR,Non Netting AP,Non Netting AR,Other,Other receipt,Payroll,Statutory contribution,Tax payable
0,2024-12-30,-1.716753e+05,2.315960e+05,-46.92314,-24639.16000,0.0,0.0000,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.0,3073.39591,-6714.28630,0.00000,-99.54311
1,2025-01-06,-6.812052e+05,4.342489e+05,-555.98095,-14851.31934,0.0,0.0000,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.0,161151.19585,-38516.01918,0.00000,-41193.93387
2,2025-01-13,-9.421128e+05,7.271149e+05,-910.65974,-49342.80475,0.0,0.0000,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,39407.03,0.0,6213.55109,-233819.56458,0.00000,-11166.59019
3,2025-01-20,-5.265080e+05,4.800459e+05,-18.90470,-14235.85073,0.0,0.0000,0.00,0.00,0.00,18996.30000,-1778159.32,265277.08,-9619.93,25265.35,0.0,894.88041,-410163.70810,0.00000,-235699.49156
4,2025-01-27,-7.408610e+05,7.058476e+06,-64.39412,-56609.76292,0.0,-678.2489,1700.14,0.00,-15178.72,187598.47000,-4496635.18,710116.20,-6890.97,14157.22,0.0,4849.80255,-117864.88000,-75126.00000,-232054.19520
5,2025-02-03,-4.496412e+05,4.366845e+05,-58.16820,-24202.45098,0.0,0.0000,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.0,2501.19064,-6818.15000,0.00000,-36509.89512
6,2025-02-10,-7.304731e+05,5.402652e+05,-865.33391,-21886.69000,0.0,-2361.1400,0.00,0.00,0.00,0.00000,0.00,0.00,-2309.16,0.00,0.0,28251.09534,-70536.56770,0.00000,-51096.31165
7,2025-02-17,-6.601685e+05,4.186628e+05,-36.73528,-34090.64631,0.0,0.0000,0.00,-794571.17,0.00,308747.14806,0.00,0.00,0.00,0.00,0.0,1788.70268,-460217.69120,0.00000,-41649.48392
8,2025-02-24,-2.303780e+06,9.069815e+06,-53.72921,-33139.55108,0.0,-91.4600,2907.41,-1896974.52,-153855.84,1206.43018,-4584572.62,1202343.61,0.00,0.00,0.0,11621.10905,-420728.52040,0.00000,-112002.60627
9,2025-03-03,-3.472537e+05,3.396383e+04,-466.06645,-40723.78805,0.0,0.0000,0.00,0.00,0.00,0.00000,0.00,0.00,0.00,0.00,0.0,44966.21434,-8112.14180,-54034.63112,-23204.51249


## 7. Weekly Cash Flow by Entity (Name)

In [13]:
# Weekly cashflow by entity
weekly_by_entity = df.groupby(['Week_Start', 'Name']).agg(
    Net_Cashflow=('Amount in USD', 'sum'),
    Transaction_Count=('Amount in USD', 'count')
).reset_index()

print("Entities:")
print(df['Name'].unique())
print(f"\nWeekly data by entity: {len(weekly_by_entity)} rows")

Entities:
['TW10' 'PH10' 'TH10' 'ID10' 'SS10' 'MY10' 'VN20' 'KR10']

Weekly data by entity: 350 rows


## 8. Export Prepared Data for Forecasting

In [14]:
# Save the weekly aggregated data
weekly_cashflow.to_csv('../processed_data/weekly_cashflow.csv', index=False)
print("✅ Saved: weekly_cashflow.csv")

weekly_pivot.to_csv('../processed_data/weekly_cashflow_by_category.csv', index=False)
print("✅ Saved: weekly_cashflow_by_category.csv")

weekly_by_entity.to_csv('../processed_data/weekly_cashflow_by_entity.csv', index=False)
print("✅ Saved: weekly_cashflow_by_entity.csv")

# Save full data with week columns
df.to_csv('../processed_data/transactions_with_weeks.csv', index=False)
print("✅ Saved: transactions_with_weeks.csv")

✅ Saved: weekly_cashflow.csv
✅ Saved: weekly_cashflow_by_category.csv
✅ Saved: weekly_cashflow_by_entity.csv
✅ Saved: transactions_with_weeks.csv


---

# 📋 Suggestions for Net Cash Flow Forecasting

## Data Prepared:
1. **`weekly_cashflow.csv`** - Main file for forecasting with columns:
   - `Week_Start`: Start date of each week
   - `Inflow`: Total positive cash flow
   - `Outflow`: Total negative cash flow  
   - `Net_Cashflow`: Inflow + Outflow (net position)
   - `Transaction_Count`: Number of transactions

2. **`weekly_cashflow_by_category.csv`** - For category-level forecasting

3. **`weekly_cashflow_by_entity.csv`** - For entity-level forecasting

## Recommendations for Forecasting:

### 1. **Time Series Models to Try:**
   - **Simple Moving Average (SMA)** - Baseline
   - **Exponential Smoothing (ETS)** - Captures trend/seasonality
   - **ARIMA/SARIMA** - For stationary data with seasonality
   - **Prophet** - Handles holidays and weekly patterns well
   - **LSTM/Neural Networks** - For complex patterns

### 2. **Feature Engineering Ideas:**
   - Week of month (1st, 2nd, 3rd, 4th week)
   - Month-end indicator (last week of month)
   - Quarter indicator
   - Lag features (previous week's cashflow)
   - Rolling averages (4-week, 8-week)
   - Category mix ratios

### 3. **Forecast Horizons:**
   - **Short-term**: 1-4 weeks ahead
   - **Medium-term**: 1-3 months ahead (4-12 weeks)
   - **Long-term**: 6 months ahead (26 weeks)

### 4. **Evaluation Metrics:**
   - MAE (Mean Absolute Error)
   - RMSE (Root Mean Square Error)
   - MAPE (Mean Absolute Percentage Error)
   - R² Score

### 5. **Additional Steps:**
   - Check for outliers/anomalies in weekly data
   - Handle missing weeks (if any) with interpolation
   - Consider separate models for inflows vs outflows
   - Test stationarity with ADF test